## install moveit_commander

```
sudo apt-get install ros-"DISTRO"-moveit-commander
pip install pyassimp
```

## reference

https://ros-planning.github.io/moveit_tutorials/doc/move_group_python_interface/move_group_python_interface_tutorial.html

## Before Running Moveit 

```
# run gazebo
roslaunch ur_gazebo ur5.launch
# run moveit
roslaunch ur5_moveit_config ur5_moveit_planning_execution.launch sim:=true limited:=true
# run rviz
roslaunch ur5_moveit_config moveit_rviz.launch config:=true
```


In [1]:
import os
ROS_DISTRO = os.environ['ROS_DISTRO']

In [2]:
from __future__ import print_function
from six.moves import input

import sys
ros_python_path = '/opt/ros/{}/lib/python2.7/dist-packages'.format(ROS_DISTRO)
if not ros_python_path in sys.path:
    sys.path.append(ros_python_path)

import rospy
import copy
import numpy as np

In [3]:
from moveit_commander.conversions import pose_to_list
from std_msgs.msg import String
import geometry_msgs.msg
import moveit_commander
import moveit_msgs.msg
from math import pi

## initialize moveit_commander

In [4]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial', anonymous=True)

In [5]:
# Provides information such as the robot’s kinematic model and the robot’s current joint states
robot = moveit_commander.RobotCommander()

In [6]:
#'''
print(robot.get_group_names())
print(robot.get_current_state())
print(robot.get_joint_names())
#'''

['endeffector', 'manipulator']
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "/world"
  name: 
    - shoulder_pan_joint
    - shoulder_lift_joint
    - elbow_joint
    - wrist_1_joint
    - wrist_2_joint
    - wrist_3_joint
  position: [-1.9632471243487757, -2.1133721510516565, 1.4760456085205078, -0.8948033491717737, -1.584893051777975, -2.16872483888735]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "/world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False
['ASSUMED_FIXED_ROOT_JOINT', 'world_joint', 'base_link-base_fixed_joint', 'shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint', 'ee_fixed_joint', 'wrist_3_passive_joint']


In [7]:
# provides a remote interface for getting, setting, and updating the robot’s internal understanding of the surrounding world
# 플래닝할때, 주변 장애물을 반영할 수 있게, 아래의 object를 통해 환경에 장애물을 추가할 수 있다.
scene = moveit_commander.PlanningSceneInterface()

In [8]:
# plan 한 trajectory를 rviz화면에 시뮬레이션해줌.
# "roslaunch ur5_moveit_config moveit_rviz.launch config:=true" 을 통해 미리 rviz를 띄어놔야 함.
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                               moveit_msgs.msg.DisplayTrajectory,
                                               queue_size=20)

In [9]:
# move_group 객체를 통해 각 joint(move_element)의 desired states를 지정할 수 있고, moveit은 이 값을 바탕으로 planning을 해줌.
group_name = "manipulator"
move_group = moveit_commander.MoveGroupCommander(group_name)

In [10]:
# We can get the name of the reference frame for this robot:
planning_frame = move_group.get_planning_frame()
print("============ Planning frame: %s" % planning_frame)

# We can also print the name of the end-effector link for this group:
eef_link = move_group.get_end_effector_link()
print("============ End effector link: %s" % eef_link)

# We can get a list of all the groups in the robot:
group_names = robot.get_group_names()
print("============ Available Planning Groups:", robot.get_group_names())

# Sometimes for debugging it is useful to print the entire state of the
# robot:
print("============ Printing robot state")
print(robot.get_current_state())
print("")

============ Planning frame: /world
============ End effector link: ee_link
============ Available Planning Groups: ['endeffector', 'manipulator']
============ Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "/world"
  name: 
    - shoulder_pan_joint
    - shoulder_lift_joint
    - elbow_joint
    - wrist_1_joint
    - wrist_2_joint
    - wrist_3_joint
  position: [-1.9632471243487757, -2.1133363882647913, 1.476069450378418, -0.8948152701007288, -1.5848091284381312, -2.168760601674215]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "/world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False



In [16]:
joint_goal = move_group.get_current_joint_values()
joint_goal[0] += 0 #pi/12
joint_goal[1] += 0
joint_goal[2] += 0
joint_goal[3] += 0
joint_goal[4] += 0
joint_goal[5] -= pi/4 

# The go command can be called with joint values, poses, or without any
# parameters if you have already set the pose or joint target for the group
move_group.go(joint_goal, wait=True)

# Calling ``stop()`` ensures that there is no residual movement
move_group.stop()

print(joint_goal)

[-1.9631989637957972, -2.113516632710592, 1.476273536682129, -0.894970719014303, -1.5849650541888636, -2.9540748417318197]


In [45]:
joint_goal = move_group.get_current_joint_values()
print(joint_goal)

[-3.0134838263141077, -1.9070351759540003, 1.0779051780700684, -0.7090609709369105, -1.5765627066241663, -0.8663600126849573]


In [46]:
move_group.get_current_pose().pose

position: 
  x: -0.201030175701
  y: -0.135452870913
  z: 0.69424337095
orientation: 
  x: 0.632883418506
  y: 0.335004131254
  z: -0.609893826168
  w: 0.339500119921

In [55]:
scale = 0.2

waypoints = []

wpose = move_group.get_current_pose().pose
wpose.position.z -= scale * 0.1  # First move up (z)
wpose.position.y += scale * 0.2  # and sideways (y)
waypoints.append(copy.deepcopy(wpose))

wpose.position.x += scale * 0.1  # Second move forward/backwards in (x)
waypoints.append(copy.deepcopy(wpose))

wpose.position.y -= scale * 0.1  # Third move sideways (y)
waypoints.append(copy.deepcopy(wpose))

# We want the Cartesian path to be interpolated at a resolution of 1 cm
# which is why we will specify 0.01 as the eef_step in Cartesian
# translation.  We will disable the jump threshold by setting it to 0.0,
# ignoring the check for infeasible jumps in joint space, which is sufficient
# for this tutorial.
(plan, fraction) = move_group.compute_cartesian_path(
                                   waypoints,   # waypoints to follow
                                   0.01,        # eef_step
                                   0.0)         # jump_threshold

In [56]:
display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = robot.get_current_state()
display_trajectory.trajectory.append(plan)
# Publish
display_trajectory_publisher.publish(display_trajectory);

In [57]:
move_group.execute(plan, wait=True)

True

## robotiq finger

In [27]:
# for gripper
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
import actionlib

In [25]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
print(action_name)

command_robotiq_action


In [28]:
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()
print("Client test: Starting sending goals")

Client test: Starting sending goals


In [44]:
## Manually set all the parameters of the gripper goal state.
######################################################################################

goal = CommandRobotiqGripperGoal()
goal.emergency_release = False
goal.stop = False
goal.position = 0.1
goal.speed = 0.1
goal.force = 5.0

# Sends the goal to the gripper.
robotiq_client.send_goal(goal)
# Block processing thread until gripper movement is finished, comment if waiting is not necesary.
robotiq_client.wait_for_result()

True

In [43]:
## Manually set all the parameters of the gripper goal state.
######################################################################################

goal = CommandRobotiqGripperGoal()
goal.emergency_release = False
goal.stop = False
goal.position = 0.0
goal.speed = 0.1
goal.force = 5.0

# Sends the goal to the gripper.
robotiq_client.send_goal(goal)
# Block processing thread until gripper movement is finished, comment if waiting is not necesary.
robotiq_client.wait_for_result()

True

In [53]:
move_group.set_max_velocity_scaling_factor = 0.001
joint_goal = move_group.get_current_joint_values()
joint_goal[0] += 0
joint_goal[1] += 0
joint_goal[2] += np.pi/12.0
joint_goal[3] -= np.pi/12.0
joint_goal[4] += 0
joint_goal[5] += 0

# The go command can be called with joint values, poses, or without any
# parameters if you have already set the pose or joint target for the group
move_group.go(joint_goal, wait=True)

# Calling ``stop()`` ensures that there is no residual movement
move_group.stop()

In [54]:
move_group.set_max_velocity_scaling_factor

0.001

In [ ]:
move_group.s